In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master="local", appName="solucionRetoDeportistasBruce")

In [4]:
path = "/home/sparky/Documentos/curso-apache-spark-platzi/files/"

In [5]:
equiposOlimpicosRDD = sc.textFile(path+"paises.csv") \
    .map(lambda line : line.split(","))

In [6]:
equiposOlimpicosRDD.take(10)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

In [7]:
deportistaOlimpicoRDD = sc.textFile(path+"deportista.csv") \
    .map(lambda l : l.split(","))
deportistaOlimpicoRDD2 = sc.textFile(path+"deportista2.csv") \
    .map(lambda l : l.split(","))
deportistaOlimpicoRDD = deportistaOlimpicoRDD \
    .union(deportistaOlimpicoRDD2)

In [8]:
deportistaOlimpicoRDD.count()

135572

In [9]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [10]:
resultadoRDD = sc.textFile(path+"resultados.csv") \
    .map(lambda l:l.split(","))

In [11]:
resultadoGanador = resultadoRDD.filter(lambda l : 'NA' not in l[1])

In [12]:
resultadoGanador.take(10)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14'],
 ['42', 'Bronze', '17', '17', '21'],
 ['43', 'Gold', '17', '17', '22'],
 ['45', 'Gold', '17', '17', '24'],
 ['49', 'Gold', '17', '17', '28'],
 ['51', 'Bronze', '17', '19', '22']]

In [13]:
deportistaGanador = resultadoGanador.map(lambda l: (l[2],l[1])) \
    .join(deportistaOlimpicoRDD.map(lambda l : (l[0],l[0:])))

In [14]:
deportistaGanador.take(10)

[('4', ('Gold', ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'])),
 ('16',
  ('Bronze',
   ['16', 'Juhamatti Tapio Aaltonen', '1', '28', '184', '85', '350'])),
 ('20', ('Gold', ['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85', '742'])),
 ('20',
  ('Bronze', ['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85', '742'])),
 ('20',
  ('Silver', ['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85', '742'])),
 ('20',
  ('Bronze', ['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85', '742'])),
 ('20',
  ('Silver', ['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85', '742'])),
 ('20', ('Gold', ['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85', '742'])),
 ('20', ('Gold', ['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85', '742'])),
 ('20', ('Gold', ['20', 'Kjetil Andr Aamodt', '1', '20', '176', '85', '742']))]

In [15]:
resultadoFinalRDD= deportistaGanador \
    .map(lambda l:(l[1][1][-1],l[1])) \
    .join(equiposOlimpicosRDD.map(lambda l:(l[0],l[2])))

In [16]:
resultadoFinalRDD.takeSample(False,6,25)

[('874',
  (('Bronze',
    ['94140',
     'Nadezhda Viktorovna Nadiya Petrova',
     '2',
     '22',
     '178',
     '65',
     '874']),
   'RUS')),
 ('1096',
  (('Gold', ['23661', 'Ralph Cook Craig', '1', '22', '182', '73', '1096']),
   'USA')),
 ('978',
  (('Gold', ['90333', 'Elvra Ozolia Lse ', '2', '20', '176', '67', '978']),
   'URS')),
 ('477',
  (('Gold', ['39706', 'Oszkr Gerde', '1', '24', '0', '0', '477']), 'HUN')),
 ('716',
  (('Gold',
    ['56071', 'Richard John Dick Joyce', '1', '22', '194', '89', '716']),
   'NZL')),
 ('1096',
  (('Gold',
    ['100529', 'Dorothy Gay Dot Richardson', '2', '34', '175', '60', '1096']),
   'USA'))]

In [23]:
valoresMedallas = {
    'Gold' : 7,
    'Silver' : 5,
    'Bronze' : 4
}

In [26]:
paisesMedallas = resultadoFinalRDD \
    .map(lambda l : (l[1][1],valoresMedallas[l[1][0][0]]))

In [27]:
from operator import add
conclusionRDD = paisesMedallas.reduceByKey((add)). \
sortBy(lambda x : x[1], ascending = False)

In [28]:
conclusionRDD.take(10)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]

In [3]:
sc.stop()